### Realization of a Multilayer Perceptron

In [1]:
from sklearn import datasets
from sklearn.preprocessing import label_binarize
import numpy as np
import tensorflow as tf

In [2]:
raw_iris = datasets.load_iris()
iris = np.column_stack([raw_iris['data'], raw_iris['target']])   # arranging the data the right way
np.random.shuffle(iris)
#print(iris[:3])   # just to take a look

target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])   # binarizing labels
#print(target_multiclass[:3]) # to take a look

In [13]:
# Parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 15
display_step = 1

# Network Parameters
n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
n_input = 4
n_classes = 3 # Iris total classes

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        total_batch = int(150/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size,:4], target_multiclass[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))

Epoch: 0001 cost=8.032277060
Epoch: 0002 cost=3.948031569
Epoch: 0003 cost=2.245248854
Epoch: 0004 cost=1.642782903
Epoch: 0005 cost=1.072124135
Epoch: 0006 cost=0.936512876
Epoch: 0007 cost=0.650709802
Epoch: 0008 cost=0.471421173
Epoch: 0009 cost=0.387785703
Epoch: 0010 cost=0.320210095
Epoch: 0011 cost=0.258614393
Epoch: 0012 cost=0.214987458
Epoch: 0013 cost=0.186933640
Epoch: 0014 cost=0.166866574
Epoch: 0015 cost=0.150772437
Epoch: 0016 cost=0.137823983
Epoch: 0017 cost=0.127687481
Epoch: 0018 cost=0.119437653
Epoch: 0019 cost=0.112400975
Epoch: 0020 cost=0.106400112
Epoch: 0021 cost=0.101237020
Epoch: 0022 cost=0.096676693
Epoch: 0023 cost=0.092615797
Epoch: 0024 cost=0.088979621
Epoch: 0025 cost=0.085677088
Optimization Finished!
Accuracy: 0.92
